In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')

from utils import edhrec_requests as edh
from utils import funciones
from utils import api_requests as req

In [3]:
req.hacer_request()

Descargando desde: https://data.scryfall.io/default-cards/default-cards-20250328090802.json
Archivo guardado como 'scryfall_cards.csv'


In [2]:
#Primera parte. Una vez tenemos el Data Frame de las Cartas, vamos a mirarlo y luego a limpiarlo
#Lo cargamos desde el fichero
df = pd.read_csv("../data/scryfall_cards.csv")
#Lo miramos para ver que está bonito
df.info()

C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\3727961735.py:3: DtypeWarning: Columns (74,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/scryfall_cards.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105911 entries, 0 to 105910
Data columns (total 87 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   object               105911 non-null  object 
 1   id                   105911 non-null  object 
 2   oracle_id            105837 non-null  object 
 3   multiverse_ids       105911 non-null  object 
 4   mtgo_id              54222 non-null   float64
 5   arena_id             16139 non-null   float64
 6   tcgplayer_id         90762 non-null   float64
 7   name                 105911 non-null  object 
 8   lang                 105911 non-null  object 
 9   released_at          105911 non-null  object 
 10  uri                  105911 non-null  object 
 11  scryfall_uri         105911 non-null  object 
 12  layout               105911 non-null  object 
 13  highres_image        105911 non-null  bool   
 14  image_status         105911 non-null  object 
 15  image_uris       

In [ ]:
#Ahora vamos a quedarnos sólo con las que son legales en Commander. 
# La columna de "legalities" contiene un diccionario en forma de string en la celda.
#Lo que me interesa es que sea "'commander': 'legal'". Con las comillas simples.
df_commander = df[df["legalities"].str.contains("'commander': 'legal'", case=False, na=False)]

#Seleccionamos las columnas que me apetecen
df_clean = df_commander[[
    "name", "type_line", "set", "set_name", "released_at", "rarity", "reprint", "edhrec_rank", "legalities", "cmc", "color_identity", "colors"    
]]

#Creamos una columna nueva con precios, que también es un diccionario en string
df_clean["eur"] = df["prices"].apply(lambda x: funciones.extraer_precio(x, "eur"))
# Convertir los precios a float, si no son None
df_clean["eur"] = pd.to_numeric(df_clean["eur"], errors='coerce')

df_clean.info()
df_clean.head(10)


<class 'pandas.core.frame.DataFrame'>
Index: 96387 entries, 0 to 105910
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            96387 non-null  object 
 1   type_line       96319 non-null  object 
 2   set             96387 non-null  object 
 3   set_name        96387 non-null  object 
 4   released_at     96387 non-null  object 
 5   rarity          96387 non-null  object 
 6   reprint         96387 non-null  bool   
 7   edhrec_rank     91886 non-null  float64
 8   legalities      96387 non-null  object 
 9   cmc             96319 non-null  float64
 10  color_identity  96387 non-null  object 
 11  colors          95132 non-null  object 
 12  eur             66588 non-null  float64
dtypes: bool(1), float64(3), object(9)
memory usage: 9.7+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\3798162672.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['eur'] = df["prices"].apply(lambda x: funciones.extraer_precio(x, "eur"))
C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\3798162672.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['eur'] = pd.to_numeric(df_clean['eur'], errors='coerce')


,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur
0,Forest,Basic Land — Forest,blb,Bloomburrow,2024-08-02,common,True,NaN,"{'standard': 'legal', 'future': 'legal', 'hist...",0.0,['G'],[],NaN
1,Fury Sliver,Creature — Sliver,tsp,Time Spiral,2006-10-06,uncommon,False,8980.0,"{'standard': 'not_legal', 'future': 'not_legal...",6.0,['R'],['R'],0.17
2,Kor Outfitter,Creature — Kor Soldier,zen,Zendikar,2009-10-02,common,False,18489.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['W'],['W'],0.22
4,Siren Lookout,Creature — Siren Pirate,xln,Ixalan,2017-09-29,common,False,16747.0,"{'standard': 'not_legal', 'future': 'not_legal...",3.0,['U'],['U'],0.11
5,Web,Enchantment — Aura,3ed,Revised Edition,1994-04-11,rare,True,24610.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['G'],['G'],0.99
6,Wastewood Verge,Land,pdft,Aetherdrift Promos,2025-02-14,rare,True,6462.0,"{'standard': 'legal', 'future': 'legal', 'hist...",0.0,"['B', 'G']",[],NaN
7,Surge of Brilliance,Instant,who,Doctor Who,2023-10-13,uncommon,False,6582.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['U'],['U'],0.20
8,Obyra's Attendants // Desperate Parry,Creature — Faerie Wizard // Instant — Adventure,woe,Wilds of Eldraine,2023-09-08,common,False,15859.0,"{'standard': 'legal', 'future': 'legal', 'hist...",5.0,['U'],['U'],0.07
9,Venerable Knight,Creature — Human Knight,eld,Throne of Eldraine,2019-10-04,uncommon,False,17004.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['W'],['W'],0.12
10,Wildcall,Sorcery,ugin,Ugin's Fate,2015-01-17,rare,True,17053.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['G'],['G'],1.57


In [4]:
#Ahora vamos a ir eliminando los tipos de carta o las cartas que no son relevantes para el análisis
#Aquí vamos a quitar las cartas de tipo "Basic Land", "Atracciones", "Tokens" y demás.
mask = df_clean[df_clean["type_line"].str.contains("Basic Land", na=False)]
maskSnow = df_clean[df_clean["type_line"].str.contains("Basic Snow Land", na=False)]
df_clean.drop(maskSnow.index, inplace=True)
df_clean.drop(mask.index, inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91960 entries, 1 to 105910
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            91960 non-null  object 
 1   type_line       91892 non-null  object 
 2   set             91960 non-null  object 
 3   set_name        91960 non-null  object 
 4   released_at     91960 non-null  object 
 5   rarity          91960 non-null  object 
 6   reprint         91960 non-null  bool   
 7   edhrec_rank     91886 non-null  float64
 8   legalities      91960 non-null  object 
 9   cmc             91892 non-null  float64
 10  color_identity  91960 non-null  object 
 11  colors          90705 non-null  object 
 12  eur             63908 non-null  float64
dtypes: bool(1), float64(3), object(9)
memory usage: 9.2+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\31274786.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(maskSnow.index, inplace=True)
C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\31274786.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(mask.index, inplace=True)


In [5]:
#Ahora vamos a quitar las Atracciones
mask = df_clean[df_clean["type_line"].str.contains("Attraction", na=False)]
df_clean.drop(mask.index, inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91873 entries, 1 to 105910
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            91873 non-null  object 
 1   type_line       91805 non-null  object 
 2   set             91873 non-null  object 
 3   set_name        91873 non-null  object 
 4   released_at     91873 non-null  object 
 5   rarity          91873 non-null  object 
 6   reprint         91873 non-null  bool   
 7   edhrec_rank     91799 non-null  float64
 8   legalities      91873 non-null  object 
 9   cmc             91805 non-null  float64
 10  color_identity  91873 non-null  object 
 11  colors          90618 non-null  object 
 12  eur             63907 non-null  float64
dtypes: bool(1), float64(3), object(9)
memory usage: 9.2+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\4236453910.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(mask.index, inplace=True)


In [6]:
#Ahora las promo de doble cara, que da la casualidad de que son las que tienen el "type_line" a False
#Igualmuente, compruebo aquí también que es así.
df_clean[df_clean["type_line"].isna()].head()

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur
668,"Jinnie Fay, Jetmir's Second // Jinnie Fay, Jet...",NaN,sld,Secret Lair Drop,2024-01-22,rare,True,3157.0,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,"['G', 'R', 'W']",NaN,NaN
1921,"Ghalta, Primal Hunger // Ghalta, Primal Hunger",NaN,sld,Secret Lair Drop,2022-12-02,rare,True,514.0,"{'standard': 'legal', 'future': 'legal', 'hist...",NaN,['G'],NaN,NaN
1927,Tuvasa the Sunlit // Tuvasa the Sunlit,NaN,sld,Secret Lair Drop,2024-12-09,mythic,True,6811.0,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,"['G', 'U', 'W']",NaN,NaN
2506,Reckoner Bankbuster // Reckoner Bankbuster,NaN,sld,Secret Lair Drop,2025-02-10,rare,True,3089.0,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,[],NaN,NaN
3605,Stitch in Time // Stitch in Time,NaN,sld,Secret Lair Drop,2022-04-22,rare,True,5046.0,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,"['R', 'U']",NaN,NaN


In [ ]:
#Y aquí las elimino, me quito los nan directamente
df_clean.dropna(subset=["type_line"], how='all', inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91805 entries, 1 to 105910
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            91805 non-null  object 
 1   type_line       91805 non-null  object 
 2   set             91805 non-null  object 
 3   set_name        91805 non-null  object 
 4   released_at     91805 non-null  object 
 5   rarity          91805 non-null  object 
 6   reprint         91805 non-null  bool   
 7   edhrec_rank     91736 non-null  float64
 8   legalities      91805 non-null  object 
 9   cmc             91805 non-null  float64
 10  color_identity  91805 non-null  object 
 11  colors          90618 non-null  object 
 12  eur             63898 non-null  float64
dtypes: bool(1), float64(3), object(9)
memory usage: 9.2+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\2840991014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.dropna(subset=['type_line'], how='all', inplace=True)


In [8]:
#Ahora vamos a mirar los NaN que hay en edhrank
df_clean[df_clean["edhrec_rank"].isna()]
#Explico por aquí como pueda, pero las cartas que salen aquí no tienen relevancia
#ya que son cartas que sólo existen en la partida gracias a otras cartas. Algunas, por ejemplo
#son el dorso de otras o el fruto de juntar otras o directamente son una mecánica que se introduce sin necesidad
#de estar en el mazo

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur
620,Carnival Elephant Meteor,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,[],[],0.14
906,"Mishra, Lost to Phyrexia",Legendary Artifact Creature — Phyrexian Artificer,bro,The Brothers' War,2022-11-18,mythic,False,NaN,"{'standard': 'legal', 'future': 'legal', 'hist...",7.0,"['B', 'R']","['B', 'R']",NaN
2043,Unhinged Beast Hunt,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,[],[],0.11
5468,"Brisela, Voice of Nightmares",Legendary Creature — Eldrazi Angel,sld,Secret Lair Drop,2023-08-10,mythic,True,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",11.0,['W'],[],NaN
8472,Squishy Sphinx Ninja,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,[],[],0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97548,"Brisela, Voice of Nightmares",Legendary Creature — Eldrazi Angel,v17,From the Vault: Transform,2017-11-24,mythic,True,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",11.0,['W'],[],NaN
97975,Spooky Clown Mox,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,[],[],0.12
100151,Urza's Dark Cannonball,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,[],[],0.18
103480,Wild Ogre Bupkis,Stickers,sunf,Unfinity Sticker Sheets,2022-10-07,common,False,NaN,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,[],[],0.11


In [9]:
#En las siguientes comprobaciones de Nan de edhrank me doy cuenta de que ninguna de esas entran dentro
#de la construcción de mazo, asi que a pesar de ser legales me las voy a fumar
df_clean.dropna(subset=["edhrec_rank"], how="all", inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91736 entries, 1 to 105910
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            91736 non-null  object 
 1   type_line       91736 non-null  object 
 2   set             91736 non-null  object 
 3   set_name        91736 non-null  object 
 4   released_at     91736 non-null  object 
 5   rarity          91736 non-null  object 
 6   reprint         91736 non-null  bool   
 7   edhrec_rank     91736 non-null  float64
 8   legalities      91736 non-null  object 
 9   cmc             91736 non-null  float64
 10  color_identity  91736 non-null  object 
 11  colors          90549 non-null  object 
 12  eur             63848 non-null  float64
dtypes: bool(1), float64(3), object(9)
memory usage: 9.2+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\358794874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.dropna(subset=["edhrec_rank"], how="all", inplace=True)


In [10]:
#Ahora voy a comprobar si hay tokens en la tabla
df_clean[df_clean["type_line"].str.contains("Token")]
#No queda ninguno asi que en principio tenemos el df limpio. Igualmente voy a mirar más cosas como los precios nulos

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur


In [11]:
#Ahora voy a mirar los duplicados y tal
df_clean[df_clean["name"].duplicated()].sort_values("name")
#Hay varias repetidas que son redundantes. 
#Creo que me voy a crear otro df para los últimos reprints y otro con las únicas para las primeras impresiones

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur
105439,"""Lifetime"" Pass Holder",Creature — Zombie Guest,unf,Unfinity,2022-10-07,rare,False,10532.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['B'],['B'],0.42
64887,A Good Day to Pie,Instant,unf,Unfinity,2022-10-07,common,False,24283.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['W'],['W'],0.09
33005,A Tale for the Ages,Enchantment,woe,Wilds of Eldraine,2023-09-08,rare,False,11563.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,['W'],['W'],NaN
43318,A Tale for the Ages,Enchantment,pwoe,Wilds of Eldraine Promos,2023-09-08,rare,True,11563.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,['W'],['W'],NaN
83699,A Tale for the Ages,Enchantment,woe,Wilds of Eldraine,2023-09-08,rare,False,11563.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,['W'],['W'],0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72117,"Éowyn, Fearless Knight",Legendary Creature — Human Knight,ltr,The Lord of the Rings: Tales of Middle-earth,2023-06-23,rare,False,2538.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,"['R', 'W']","['R', 'W']",0.34
101096,"Éowyn, Lady of Rohan",Legendary Creature — Human Noble,ltr,The Lord of the Rings: Tales of Middle-earth,2023-11-03,uncommon,True,6511.0,"{'standard': 'not_legal', 'future': 'not_legal...",3.0,['W'],['W'],NaN
59635,"Éowyn, Shieldmaiden",Legendary Creature — Human Knight,ltc,Tales of Middle-earth Commander,2023-11-03,mythic,True,3485.0,"{'standard': 'not_legal', 'future': 'not_legal...",5.0,"['R', 'U', 'W']","['R', 'U', 'W']",NaN
9072,"Éowyn, Shieldmaiden",Legendary Creature — Human Knight,ltc,Tales of Middle-earth Commander,2023-06-23,mythic,False,3485.0,"{'standard': 'not_legal', 'future': 'not_legal...",5.0,"['R', 'U', 'W']","['R', 'U', 'W']",1.23


In [12]:
#Vamos a limpiar los duplicados

#convierto la columna de released_at 
df_clean["released_at"] = pd.to_datetime(df_clean["released_at"])

#Ordenamos el DataFrame por "released_at" de forma ascendente para tener los viejos primero
df_clean = df_clean.sort_values("released_at")

#elimino duplicados mirando el nombre y quedándome con el primero que aparezca por antigüedad
df_final = df_clean.drop_duplicates(subset=["name"], keep="first")

#y ahora compruebo que df_final está bien
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28369 entries, 58595 to 6875
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            28369 non-null  object        
 1   type_line       28369 non-null  object        
 2   set             28369 non-null  object        
 3   set_name        28369 non-null  object        
 4   released_at     28369 non-null  datetime64[ns]
 5   rarity          28369 non-null  object        
 6   reprint         28369 non-null  bool          
 7   edhrec_rank     28369 non-null  float64       
 8   legalities      28369 non-null  object        
 9   cmc             28369 non-null  float64       
 10  color_identity  28369 non-null  object        
 11  colors          27935 non-null  object        
 12  eur             25196 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(3), object(8)
memory usage: 2.8+ MB


C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\3402230447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["released_at"] = pd.to_datetime(df_clean["released_at"])


In [13]:
#Y con esto termino la limpieza de las cartas y un último vistazo
df_final

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur
58595,Jade Statue,Artifact,lea,Limited Edition Alpha,1993-08-05,uncommon,False,26972.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,[],[],232.12
45854,Plateau,Land — Mountain Plains,lea,Limited Edition Alpha,1993-08-05,rare,False,478.0,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,"['R', 'W']",[],3273.87
55849,Disintegrate,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,14349.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['R'],['R'],16.08
30496,White Ward,Enchantment — Aura,lea,Limited Edition Alpha,1993-08-05,uncommon,False,23323.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['W'],['W'],35.67
1987,Sinkhole,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,16511.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['B'],['B'],83.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32523,Stall Out,Sorcery,dft,Aetherdrift,2025-02-14,common,False,17581.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,['U'],['U'],0.11
32417,Kickoff Celebrations,Enchantment,dft,Aetherdrift,2025-02-14,common,False,14039.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,['R'],['R'],0.07
32423,Endrider Spikespitter,Creature — Human Mercenary,dft,Aetherdrift,2025-02-14,uncommon,False,19471.0,"{'standard': 'legal', 'future': 'legal', 'hist...",4.0,['R'],['R'],0.08
104528,Road Rage,Instant,dft,Aetherdrift,2025-02-14,uncommon,False,19248.0,"{'standard': 'legal', 'future': 'legal', 'hist...",1.0,['R'],['R'],0.11


In [14]:
#Ahora vamos a añadir una columna de años de publicación
df_final = edh.gestiona_el_Release(df_final)
df_final.info()
df_final.head()

<class 'pandas.core.frame.DataFrame'>
Index: 28369 entries, 58595 to 6875
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            28369 non-null  object        
 1   type_line       28369 non-null  object        
 2   set             28369 non-null  object        
 3   set_name        28369 non-null  object        
 4   released_at     28369 non-null  datetime64[ns]
 5   rarity          28369 non-null  object        
 6   reprint         28369 non-null  bool          
 7   edhrec_rank     28369 non-null  float64       
 8   legalities      28369 non-null  object        
 9   cmc             28369 non-null  float64       
 10  color_identity  28369 non-null  object        
 11  colors          27935 non-null  object        
 12  eur             25196 non-null  float64       
 13  release_year    28369 non-null  int32         
dtypes: bool(1), datetime64[ns](1), float64(3), int32(1), obj

e:\AreaTrabajo\REPO_PRUEBA\Itsazain_Bilbao_EDA\src\notebooks\..\utils\edhrec_requests.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame["released_at"] = pd.to_datetime(dataFrame["released_at"], errors="coerce")
e:\AreaTrabajo\REPO_PRUEBA\Itsazain_Bilbao_EDA\src\notebooks\..\utils\edhrec_requests.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame["release_year"] = dataFrame["released_at"].dt.year


,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur,release_year
58595,Jade Statue,Artifact,lea,Limited Edition Alpha,1993-08-05,uncommon,False,26972.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,[],[],232.12,1993
45854,Plateau,Land — Mountain Plains,lea,Limited Edition Alpha,1993-08-05,rare,False,478.0,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,"['R', 'W']",[],3273.87,1993
55849,Disintegrate,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,14349.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['R'],['R'],16.08,1993
30496,White Ward,Enchantment — Aura,lea,Limited Edition Alpha,1993-08-05,uncommon,False,23323.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['W'],['W'],35.67,1993
1987,Sinkhole,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,16511.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['B'],['B'],83.90,1993


In [ ]:
#Y ahora lo voy a pasar a un .csv para no tener que hacer eta puñetera limpieza todo el rato cada vez que reinicio el pc
#Importante: como le fui añadiendo y quitando columnas, pero esto no lo he movido de sitio (me da miedo moverlo y romperlo)
#esto sigue aquí, pero hay que hacer esto al final cuando ya se ha limpiado e imputado todo.
df_final.to_csv("../data/cartas_limpiadas.csv", index=False)

In [15]:
#Vale, lo siguiente con esto es imputar los colors con el color idéntity para los que lo tengan null porque son de
#doble cara. Y después separar los colores en columnas diferentes con true or false de valor.
df_final[df_final.colors.isna()]

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur,release_year
43269,Mayor of Avabruck // Howlpack Alpha,Creature — Human Advisor Werewolf // Creature ...,pisd,Innistrad Promos,2011-09-24,rare,True,7210.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['G'],NaN,NaN,2011
21654,Ludevic's Test Subject // Ludevic's Abomination,Creature — Lizard Egg // Creature — Lizard Horror,pisd,Innistrad Promos,2011-09-24,rare,True,15205.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['U'],NaN,NaN,2011
8261,Bloodline Keeper // Lord of Lineage,Creature — Vampire // Creature — Vampire,isd,Innistrad,2011-09-30,rare,False,2461.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,['B'],NaN,9.95,2011
56633,Screeching Bat // Stalking Vampire,Creature — Bat // Creature — Vampire,isd,Innistrad,2011-09-30,uncommon,False,21850.0,"{'standard': 'not_legal', 'future': 'not_legal...",3.0,['B'],NaN,0.13,2011
54184,Ulvenwald Mystics // Ulvenwald Primordials,Creature — Human Shaman Werewolf // Creature —...,isd,Innistrad,2011-09-30,uncommon,False,23401.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,['G'],NaN,0.15,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55618,Hydroelectric Specimen // Hydroelectric Labora...,Creature — Weird // Land,mh3,Modern Horizons 3,2024-06-14,uncommon,False,1278.0,"{'standard': 'not_legal', 'future': 'not_legal...",3.0,['U'],NaN,0.64,2024
2536,Waterlogged Teachings // Inundated Archive,Instant // Land,mh3,Modern Horizons 3,2024-06-14,uncommon,False,2830.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,"['B', 'U']",NaN,0.34,2024
4240,Bloodsoaked Insight // Sanguine Morass,Sorcery // Land,mh3,Modern Horizons 3,2024-06-14,uncommon,False,4320.0,"{'standard': 'not_legal', 'future': 'not_legal...",7.0,"['B', 'R']",NaN,0.18,2024
50514,Drowner of Truth // Drowned Jungle,Creature — Eldrazi // Land,mh3,Modern Horizons 3,2024-06-14,uncommon,False,6793.0,"{'standard': 'not_legal', 'future': 'not_legal...",7.0,"['G', 'U']",NaN,0.15,2024


In [16]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28369 entries, 58595 to 6875
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            28369 non-null  object        
 1   type_line       28369 non-null  object        
 2   set             28369 non-null  object        
 3   set_name        28369 non-null  object        
 4   released_at     28369 non-null  datetime64[ns]
 5   rarity          28369 non-null  object        
 6   reprint         28369 non-null  bool          
 7   edhrec_rank     28369 non-null  float64       
 8   legalities      28369 non-null  object        
 9   cmc             28369 non-null  float64       
 10  color_identity  28369 non-null  object        
 11  colors          27935 non-null  object        
 12  eur             25196 non-null  float64       
 13  release_year    28369 non-null  int32         
dtypes: bool(1), datetime64[ns](1), float64(3), int32(1), obj

In [ ]:
#Imputamos valores. 
df_final["colors"].fillna(df_final["color_identity"], inplace=True)
df_final.info()
df_final


<class 'pandas.core.frame.DataFrame'>
Index: 28369 entries, 58595 to 6875
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   name            28369 non-null  object        
 1   type_line       28369 non-null  object        
 2   set             28369 non-null  object        
 3   set_name        28369 non-null  object        
 4   released_at     28369 non-null  datetime64[ns]
 5   rarity          28369 non-null  object        
 6   reprint         28369 non-null  bool          
 7   edhrec_rank     28369 non-null  float64       
 8   legalities      28369 non-null  object        
 9   cmc             28369 non-null  float64       
 10  color_identity  28369 non-null  object        
 11  colors          28369 non-null  object        
 12  eur             25196 non-null  float64       
 13  release_year    28369 non-null  int32         
dtypes: bool(1), datetime64[ns](1), float64(3), int32(1), obj

C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\2645350157.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['colors'].fillna(df_final['color_identity'], inplace=True)
C:\Users\Edurne\AppData\Local\Temp\ipykernel_1604\2645350157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['colors'].fillna(df_final['color_identity'

,name,type_line,set,set_name,released_at,rarity,reprint,edhrec_rank,legalities,cmc,color_identity,colors,eur,release_year
58595,Jade Statue,Artifact,lea,Limited Edition Alpha,1993-08-05,uncommon,False,26972.0,"{'standard': 'not_legal', 'future': 'not_legal...",4.0,[],[],232.12,1993
45854,Plateau,Land — Mountain Plains,lea,Limited Edition Alpha,1993-08-05,rare,False,478.0,"{'standard': 'not_legal', 'future': 'not_legal...",0.0,"['R', 'W']",[],3273.87,1993
55849,Disintegrate,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,14349.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['R'],['R'],16.08,1993
30496,White Ward,Enchantment — Aura,lea,Limited Edition Alpha,1993-08-05,uncommon,False,23323.0,"{'standard': 'not_legal', 'future': 'not_legal...",1.0,['W'],['W'],35.67,1993
1987,Sinkhole,Sorcery,lea,Limited Edition Alpha,1993-08-05,common,False,16511.0,"{'standard': 'not_legal', 'future': 'not_legal...",2.0,['B'],['B'],83.90,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32523,Stall Out,Sorcery,dft,Aetherdrift,2025-02-14,common,False,17581.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,['U'],['U'],0.11,2025
32417,Kickoff Celebrations,Enchantment,dft,Aetherdrift,2025-02-14,common,False,14039.0,"{'standard': 'legal', 'future': 'legal', 'hist...",2.0,['R'],['R'],0.07,2025
32423,Endrider Spikespitter,Creature — Human Mercenary,dft,Aetherdrift,2025-02-14,uncommon,False,19471.0,"{'standard': 'legal', 'future': 'legal', 'hist...",4.0,['R'],['R'],0.08,2025
104528,Road Rage,Instant,dft,Aetherdrift,2025-02-14,uncommon,False,19248.0,"{'standard': 'legal', 'future': 'legal', 'hist...",1.0,['R'],['R'],0.11,2025
